# Relief Pitcher Summary Statistics and Rest Day Analysis

This notebook loads the filtered relief pitcher dataset, calculates rest days between appearances, and creates game-level summary statistics for analysis.

In [2]:
# Import necessary packages for data analysis
import pandas as pd
import numpy as np
import polars as pl
from datetime import datetime, timedelta

In [ ]:
# Load the relief pitcher dataset with polars
(statcast_relievers := pl.read_parquet('data/statcast_relievers_final.parquet'))

pitcher_id,date,game_id,velocity,spin_rate,pitch_type_abbr,pitch_type,pitch_result,ab_result,pitch_call,strike_zone,ball_count,strike_count,outs,inning,top_bottom,home_team,away_team,ab_number,pitch_num,batted_ball_type,runner_1b,runner_2b,runner_3b,away_score,home_score,defense_score,offense_score,game_type,batter_side,pitcher_hand,pitcher_days_since_prev_game,pitcher_days_until_next_game,year,pitcher_name
i64,datetime[ns],i64,f64,i64,str,str,str,str,str,i64,i64,i64,i64,i64,str,str,str,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i32,str
433589,2021-04-01 00:00:00,634640,85.8,1989,"""FF""","""4-Seam Fastball""","""ball""",null,"""B""",11,0,0,2,6,"""Top""","""ATH""","""HOU""",41,1,null,null,670541,null,3,0,0,3,"""R""","""R""","""R""",null,3,2021,"""Yusmeiro Petit"""
433589,2021-04-01 00:00:00,634640,87.5,2182,"""FF""","""4-Seam Fastball""","""ball""",null,"""B""",12,0,0,1,6,"""Top""","""ATH""","""HOU""",39,1,null,663656,608324,null,1,0,0,1,"""R""","""R""","""R""",null,3,2021,"""Yusmeiro Petit"""
433589,2021-04-01 00:00:00,634640,85.6,2136,"""FF""","""4-Seam Fastball""","""ball""",null,"""B""",11,0,0,2,6,"""Top""","""ATH""","""HOU""",40,1,null,663656,608324,null,1,0,0,1,"""R""","""L""","""R""",null,3,2021,"""Yusmeiro Petit"""
433589,2021-04-01 00:00:00,634640,72.4,1772,"""CU""","""Curveball""","""called_strike""",null,"""S""",5,1,0,2,6,"""Top""","""ATH""","""HOU""",41,2,null,null,670541,null,3,0,0,3,"""R""","""R""","""R""",null,3,2021,"""Yusmeiro Petit"""
433589,2021-04-01 00:00:00,634640,87.2,2153,"""FF""","""4-Seam Fastball""","""ball""",null,"""B""",14,1,0,1,6,"""Top""","""ATH""","""HOU""",39,2,null,663656,608324,null,1,0,0,1,"""R""","""R""","""R""",null,3,2021,"""Yusmeiro Petit"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
702352,2024-09-28 00:00:00,745281,94.7,1861,"""SI""","""Sinker""","""foul""",null,"""S""",4,2,2,0,9,"""Top""","""SF""","""STL""",80,5,null,695336,null,null,5,6,6,5,"""R""","""R""","""R""",3,null,2024,"""Spencer Bivens"""
702352,2024-09-28 00:00:00,745281,89.3,1631,"""CH""","""Changeup""","""blocked_ball""",null,"""B""",13,2,2,1,9,"""Top""","""SF""","""STL""",81,5,null,695336,null,null,5,6,6,5,"""R""","""L""","""R""",3,null,2024,"""Spencer Bivens"""
702352,2024-09-28 00:00:00,745281,92.3,2002,"""FC""","""Cutter""","""called_strike""","""strikeout""","""S""",12,2,2,0,9,"""Top""","""SF""","""STL""",80,6,null,695336,null,null,5,6,6,5,"""R""","""R""","""R""",3,null,2024,"""Spencer Bivens"""


In [ ]:
# Create pitch count and rest days table
(pitch_count_rest_days := (statcast_relievers
    .group_by(['pitcher_id', 'pitcher_name', 'date', 'game_id'])
    .agg(pl.col('pitch_num').count().alias('pitch_count'))
    .sort(['pitcher_id', 'date'])
    .with_columns([
        pl.col('date').diff().over('pitcher_id').dt.total_days().alias('rest_days')
    ])
    .select(['pitcher_name', 'game_id', 'date', 'pitcher_id', 'rest_days', 'pitch_count'])
))  ## what to do with null?? big day to offset?? removing it would mess up the dt function

pitcher_name,game_id,date,pitcher_id,rest_days,pitch_count
str,i64,datetime[ns],i64,i64,u32
"""Yusmeiro Petit""",634640,2021-04-01 00:00:00,433589,null,11
"""Yusmeiro Petit""",634651,2021-04-04 00:00:00,433589,3,26
"""Yusmeiro Petit""",634628,2021-04-06 00:00:00,433589,2,16
"""Yusmeiro Petit""",634567,2021-04-07 00:00:00,433589,1,10
"""Yusmeiro Petit""",634537,2021-04-09 00:00:00,433589,2,8
…,…,…,…,…,…
"""Spencer Bivens""",746985,2024-09-19 00:00:00,702352,4,37
"""Spencer Bivens""",746255,2024-09-21 00:00:00,702352,2,28
"""Spencer Bivens""",747150,2024-09-23 00:00:00,702352,2,21
